In [59]:
from langchain_openai import ChatOpenAI 
from dotenv import load_dotenv
import os
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, START, END, MessagesState 
from langchain_core.messages import SystemMessage, HumanMessage
from IPython.display import display, Image
from tools.visit_web_page_tool import VisitWebPageSyncTool
from tools.atom_feed_find_tool import AtomFeedFindTool
from tools.atom_feed_read_tool import AtomFeedReadTool
from langgraph.checkpoint.memory import MemorySaver

In [60]:
load_dotenv()

True

In [61]:
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [62]:
llm_with_tools = llm.bind_tools([VisitWebPageSyncTool(), 
                                 AtomFeedFindTool(), 
                                 AtomFeedReadTool()])

In [63]:
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [64]:
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([VisitWebPageSyncTool(), 
                                    AtomFeedFindTool(), 
                                    AtomFeedReadTool()]))
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition, "tools")
builder.add_edge("tools", "tool_calling_llm")
builder.add_edge("tools", END)

In [65]:
graph = builder.compile(checkpointer=MemorySaver())

In [66]:
display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [67]:
messages = [HumanMessage(content="visit https://www.thecloudcast.net and from the podcast archive get all podcast links.")]

In [68]:
config = {"configurable": {"thread_id": 1}}
output = graph.invoke({"messages": messages}, config=config)
for item in output["messages"]:
    item.pretty_print()

browser =  <Browser type=<BrowserType name=chromium executable_path=/Users/mahadevgaonkar/Library/Caches/ms-playwright/chromium-1161/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=134.0.6998.35>
page =  <Page url='about:blank'>
page title: The Cloudcast
================================ Human Message =================================

find atom link on https://www.thecloudcast.net and list all the links in atom feed.
================================== Ai Message ==================================
Tool Calls:
  visit_webpage (call_8oKDKdnjtUgryVAg0SDZbmuv)
 Call ID: call_8oKDKdnjtUgryVAg0SDZbmuv
  Args:
    url: https://www.thecloudcast.net
    clean_flag: True
================================= Tool Message =================================
Name: visit_webpage

Sunday, April 20, 2025 Listen to The Cloudcast Posted by Brian Gracely AI Augmenting vs. Replacing Instead of focusing on augmenting existing teams and processes with AI, are we beginning to see more companies looking to